In [3]:
import pandas as pd
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder


ml_df = pd.read_csv('../data/ml_merged.csv') 
with open('../data/matched_amazon_reviews.jsonl') as f:
    matched_data = [json.loads(line) for line in f]
matched_df = pd.DataFrame(matched_data)  

# --- Keep only one asin per movieId
matched_unique = matched_df.drop_duplicates(subset='movieId', keep='first')

# --- Merge MovieLens with unique matched asin
merged = pd.merge(ml_df, matched_unique[['movieId', 'asin']], on='movieId', how='inner')
merged = merged.dropna(subset=['asin'])

print(f"Total data setelah merge (no duplicate asin): {len(merged)}")

Total data setelah merge (no duplicate asin): 381285


In [ ]:
df_qi = pd.read_csv('../data/encoded_amazon_vectors.csv')
asin_order = df_qi['asin'].tolist()

item_encoder = LabelEncoder()
item_encoder.classes_ = np.array(asin_order)
merged['item_index'] = item_encoder.transform(merged['asin'])

# --- Label encoding userId
user_encoder = LabelEncoder()
merged['user_index'] = user_encoder.fit_transform(merged['userId'])

# --- Simpan untuk training PMF
print("Jumlah user unik:", merged['user_index'].nunique())
print("Jumlah item unik:", merged['item_index'].nunique())
merged[['user_index', 'item_index', 'rating']].to_csv('../data/pmf_training_data.csv', index=False)

C:\Users\fikra\AppData\Local\Temp\ipykernel_30404\561628164.py:2: DtypeWarning: Columns (256) have mixed types. Specify dtype option on import or set low_memory=False.
  df_qi = pd.read_csv('../data/encoded_amazon_vectors.csv')


Jumlah user unik: 6040
Jumlah item unik: 1345


In [5]:
print("Duplikasi user-item:", merged.duplicated(subset=['user_index', 'item_index']).sum())

Duplikasi user-item: 0
